In [2]:
import pandas as pd
import tensorflow as tf
import numpy as np
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification, create_optimizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tf2onnx
import onnx
import json

In [4]:
file = "intent_examples_1000_multilingual.xlsx"
df = pd.read_excel(file)

In [7]:
df['Example Query (Spanish)'] = df['Example Query (Spanish)'].str.lstrip('¿')
df.to_excel("cleaned_intent_examples_1000_multilingual.xlsx", index = False)

In [3]:
dataset = "cleaned_intent_examples_1000_multilingual.xlsx"
ds = pd.read_excel(dataset)

In [4]:
# Extracting columns
df1 = ds[['Example Query (English)', 'Example Query (Arabic)', 'Example Query (Spanish)', 'Intent Category']]
df1.columns = ['English', 'Arabic', 'Spanish', 'Category']

In [5]:
# Changing data format from wide to long
ndf = df1.melt(id_vars = ['Category'], var_name = 'Language', value_name = 'Sample')
ndf = ndf[['Sample', 'Category']]


In [6]:
ndf.to_excel("Final Samples.xlsx")

In [7]:
print(ndf.head())

                             Sample        Category
0             Play some rock music.       Media (M)
1                   Pause the song.       Media (M)
2           Skip to the next track.       Media (M)
3              Increase the volume.       Media (M)
4  Take me to the nearest pharmacy.  Navigation (N)


In [33]:
# Encoding Categories
encoder = LabelEncoder()
ndf['Category'] = encoder.fit_transform(ndf['Category'])

In [34]:
num_labels = len(encoder.classes_)
num_labels

9

In [35]:
ndf.head()

,Sample,Category
0,Play some rock music.,4
1,Pause the song.,4
2,Skip to the next track.,4
3,Increase the volume.,4
4,Take me to the nearest pharmacy.,5


In [36]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

In [37]:
def tokenize_function(samples, categories):

    encodings = tokenizer(samples.tolist(), padding = True, truncation = True, return_tensors = 'tf')
    return encodings, tf.keras.utils.to_categorical(categories, num_classes = num_labels)

In [38]:
train_samples, test_samples, train_categories, test_categories = train_test_split(ndf['Sample'], ndf['Category'], test_size = 0.2, random_state = 41, shuffle = True)
train_encodings, train_categories = tokenize_function(train_samples, train_categories)
test_encodings, test_categories = tokenize_function(test_samples, test_categories)

In [39]:
train_dataset = tf.data.Dataset.from_tensor_slices(({"input_ids": train_encodings["input_ids"], "attention_mask": train_encodings["attention_mask"]}, train_categories)).batch(16)
test_dataset = tf.data.Dataset.from_tensor_slices(({"input_ids": test_encodings["input_ids"], "attention_mask": test_encodings["attention_mask"]}, test_categories)).batch(16)

In [40]:
model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels = num_labels)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [41]:
training_steps = len(train_dataset) * 10
optimizer, schedule = create_optimizer(init_lr = 3e-5, num_warmup_steps = 0, num_train_steps = training_steps)
loss = tf.keras.losses.CategoricalCrossentropy(from_logits = True)
metrics = [tf.keras.metrics.CategoricalAccuracy(name = 'accuracy'), 
           tf.keras.metrics.Precision(name = 'precision'),
           tf.keras.metrics.Recall(name = 'recall')]
model.compile(optimizer = optimizer, loss = loss, metrics = metrics)

In [42]:
model.fit(train_dataset, validation_data = test_dataset, epochs = 10)

Epoch 1/10
150/150 [==============================] - 182s 1s/step - loss: 1.1731 - accuracy: 0.7358 - precision: 0.6819 - recall: 0.6733 - val_loss: 0.1344 - val_accuracy: 1.0000 - val_precision: 0.9788 - val_recall: 1.0000
Epoch 2/10
150/150 [==============================] - 155s 1s/step - loss: 0.0746 - accuracy: 1.0000 - precision: 0.9881 - recall: 1.0000 - val_loss: 0.0263 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 3/10
150/150 [==============================] - 148s 988ms/step - loss: 0.0252 - accuracy: 1.0000 - precision: 0.9992 - recall: 1.0000 - val_loss: 0.0126 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 4/10
150/150 [==============================] - 146s 976ms/step - loss: 0.0142 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.0078 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 5/10
150/150 [==============================] - 145s 965ms/step - loss: 0.0095 - accurac

In [43]:
test_loss, test_accuracy, test_precision, test_recall = model.evaluate(test_dataset)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")
print(f"Test Precision: {test_precision}")
print(f"Test Recall: {test_recall}")

38/38 [==============================] - 7s 175ms/step - loss: 0.0029 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000
Test Loss: 0.0028826287016272545
Test Accuracy: 1.0
Test Precision: 1.0
Test Recall: 1.0


In [44]:
true_labels = tf.argmax(test_categories, axis = -1).numpy()
true_labels = encoder.inverse_transform(true_labels)    

prediction = model.predict(test_dataset)

predicted_labels = tf.argmax(prediction.logits, axis = -1).numpy()
predicted_labels = encoder.inverse_transform(predicted_labels)

for i in range(10):
    print(f"Predicted: {predicted_labels[i]}, True: {true_labels[i]}")

38/38 [==============================] - 9s 184ms/step
Predicted: 2, True: 2
Predicted: 4, True: 4
Predicted: 0, True: 0
Predicted: 2, True: 2
Predicted: 3, True: 3
Predicted: 1, True: 1
Predicted: 7, True: 7
Predicted: 6, True: 6
Predicted: 4, True: 4
Predicted: 0, True: 0


In [50]:
#Testing new sample
sample = ["Can u tell me a wise saying?"]

In [51]:
sample_encoding = tokenizer(sample, padding = True, truncation = True, return_tensors = "tf")

In [52]:
predictions = model(sample_encoding)

In [53]:
probabilities = tf.nn.softmax(predictions.logits, axis = -1).numpy()
predicted = np.argmax(probabilities, axis = -1)
print(predicted)

[8]


In [54]:
spec = (tf.TensorSpec((None, 512), tf.int32, name = "input_ids"), tf.TensorSpec((None, 512), tf.int32, name = "attention_mask"))

In [55]:
onnxModel, _ = tf2onnx.convert.from_keras(model, input_signature = spec, opset = 13)

In [56]:
onnx.save(onnxModel, "intentDetection.onnx")

In [ ]:
IDT = onnx.load("intentDetection.onnx")
onnx.checker.check_model(IDT)